[slides](https://docs.google.com/presentation/d/19K9nVjuSOCrZGM6lmFeEEarTm2xZwDSiZEIzf-Ywr5o/edit?usp=sharing)

[python-ecdsa docs](https://github.com/warner/python-ecdsa)

# Signing our First Message with ECDSA

In [1]:
from ecdsa import SigningKey, SECP256k1

private_key = SigningKey.generate(curve=SECP256k1)
public_key = private_key.get_verifying_key()

message = b"Don't touch that XRP"
signature = private_key.sign(message)

In [2]:
public_key.verify(signature, message)

True

In [3]:
public_key.verify(signature, b"Nice XRP")

BadSignatureError: 

# Defining ECDSACoin

* A coin is just a list of transfers, just like with PNGCoin. 
    * Where transfers were photographs of signatures in PNGCoin, they are ECDSA digital signatures in ECDSACoin
* The `public_key` in the last transfer is who owns the coin
* To spend the coin append a new transfer. Use the public key of the person you are sending to, and sign it using your private key.

In [4]:
class Transfer:
    
    def __init__(self, signature, public_key):
        self.signature = signature
        self.public_key = public_key

class ECDSACoin:
    
    def __init__(self, transfers):
        self.transfers = transfers

In [5]:
# The usual suspects ... 
# SECP256k1 is a detail about the "magical multiplication" used under the covers

bank_private_key = SigningKey.generate(curve=SECP256k1)
bob_private_key = SigningKey.generate(curve=SECP256k1)
alice_private_key = SigningKey.generate(curve=SECP256k1)

bank_public_key = bank_private_key.get_verifying_key()
bob_public_key = bob_private_key.get_verifying_key()
alice_public_key = alice_private_key.get_verifying_key()

In [6]:
from utils import serialize

def issue(public_key):
    # Create a message specifying who the coin is being issued to
    message = serialize(public_key)
    
    # Create the first transfer, signing with the banks private key
    signature = bank_private_key.sign(message)
    transfer = Transfer(
        signature=signature,
        public_key=public_key,
    )
    
    # Create and return the coin with just the issuing transfer
    coin = ECDSACoin(transfers=[transfer])
    return coin

# Validating the First Transfer

In [7]:
def validate(coin):
    transfer = coin.transfers[0]
    message = serialize(transfer.public_key)
    assert bank_public_key.verify(transfer.signature, message)

In [8]:
# give alice a ECDSACoin
alice_coin = issue(alice_public_key)

validate(alice_coin)

In [9]:
# Example where the signature is done using wrong private key

bad_message = serialize(alice_public_key)
bad_transfer = Transfer(
    signature=bob_private_key.sign(bad_message),  # WRONG SIGNING KEY
    public_key=alice_public_key,
)
bad_coin = ECDSACoin(transfers=[bad_transfer])

In [10]:
from ecdsa import BadSignatureError

try:
    validate(bad_coin)
except BadSignatureError:
    print("Bad signature detected")

Bad signature detected


# Validating Subsequent Transfers

In [11]:
def transfer_message(previous_signature, public_key):
    return serialize({
        "previous_signature": previous_signature, 
        "next_owner_public_key": public_key,
    })

def validate(coin):
    # Check the first transfer
    transfer = coin.transfers[0]
    message = serialize(transfer.public_key)
    assert bank_public_key.verify(transfer.signature, message)
    
    # Check the subsequent transfers
    previous_transfer = coin.transfers[0]
    for transfer in coin.transfers[1:]:
        # Check previous owner signed this transfer using their private key
        assert previous_transfer.public_key.verify(
            transfer.signature,
            transfer_message(previous_transfer.signature, transfer.public_key),
        )
        previous_transfer = transfer

In [12]:
def get_owner(coin):
    database = {
        bob_public_key: "Bob",
        alice_public_key: "Alice",
        bank_public_key: "Bank",
    }
    public_key = coin.transfers[-1].public_key
    return database[public_key]

In [23]:
alice_coin = issue(alice_public_key)

print("This coin is owned by", get_owner(alice_coin))

m1 = transfer_message(alice_coin.transfers[0].signature, bob_public_key)
alice_to_bob = Transfer(
    signature=alice_private_key.sign(m1),
    public_key=bob_public_key,
)
alice_coin.transfers.append(alice_to_bob)
validate(alice_coin)

print("This coin is owned by", get_owner(alice_coin))

m2 = transfer_message(alice_coin.transfers[1].signature, bank_public_key)
bob_to_bank = Transfer(
    signature=bob_private_key.sign(m2),
    public_key=bank_public_key,
)
alice_coin.transfers.append(bob_to_bank)
validate(alice_coin)

print("This coin is owned by", get_owner(alice_coin))

This coin is owned by Alice
This coin is owned by Bob
This coin is owned by Bank


In [14]:
# No errors means the signature was good
validate(alice_coin)

In [15]:
alice_coin = issue(alice_public_key)

m = transfer_message(alice_coin.transfers[0].signature, bob_public_key)
alice_to_bob_forged = Transfer(
    signature=bob_private_key.sign(m),
    public_key=bob_public_key,
)

alice_coin.transfers.append(alice_to_bob_forged)

In [16]:
try:
    # Should raise a BadSignatureError
    validate(alice_coin)
except BadSignatureError:
    print("Bad signature detected")

Bad signature detected


# Serialization

In [17]:
# The code from last time still works (utils.py)
from utils import to_disk, from_disk

In [18]:
import os

filename = "alices.ecdsacoin"
 
print("Alice's coinfile exists on disk?", os.path.isfile(filename))

Alice's coinfile exists on disk? False


In [19]:
alice_coin = issue(alice_public_key)

to_disk(alice_coin, filename)

In [20]:
print("Alice's coinfile exists on disk?", os.path.isfile(filename))

Alice's coinfile exists on disk? True


# The Finished Product

[ecdsacoin.py](ecdsacoin.py)

In [21]:
import ecdsacoin

coin = ecdsacoin.issue(alice_public_key)
coin.validate()

message = ecdsacoin.transfer_message(coin.transfers[0].signature, bob_public_key)
alice_to_bob = Transfer(
    signature=alice_private_key.sign(message),
    public_key=bob_public_key,
)

coin.transfers.append(alice_to_bob)
coin.validate()